In [1]:
n_epochs_all = None
save_path = 'data/'
show_plot = True

import os
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from matplotlib import gridspec
from scvi.dataset import GeneExpressionDataset, CortexDataset, RetinaDataset, BrainLargeDataset, HematoDataset, CbmcDataset, SyntheticDataset, SyntheticRandomDataset, PbmcDataset, BrainSmallDataset
from scvi.models import VAE
from scvi.inference import UnsupervisedTrainer, AdapterTrainer
from tqdm import tqdm_notebook

/home/pierre/miniconda3/envs/scvi/lib/python3.6/site-packages/scikit_learn-0.19.2-py3.6-linux-x86_64.egg/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# my_dataset = BrainSmallDataset(save_path=save_path)
my_dataset = CortexDataset()
# my_dataset = HematoDataset()
my_dataset_name = 'cortexB'

save_path = '/home/pierre/scVI/results/ppc_figs'

File data/expression.bin already downloaded
Preprocessing Cortex data
Finished preprocessing Cortex data


In [7]:
# set hyperparameters 
from scvi.inference import UnsupervisedTrainer
n_epochs=120
lr=0.0004
use_batches=False
use_cuda=True
verbose = False
infer_batch_size = 32
early_stopping_kwargs = {"early_stopping_metric": "ll"}

n_samples_posterior_density= 25 # 250 is scVI-reproducibility

nb_simu = 20

phi_name = 'ratio_zero_mean'
def phi(array, axis=None):
    nb_zeros = (array.astype(int) == 0).sum(axis=axis)
    
    nb_non_zeros = (array.astype(int) != 0).sum(axis=axis)
    avg_expression = array.sum(axis=axis) / nb_non_zeros
    
    return nb_zeros / avg_expression

In [8]:
from scipy.stats.mstats import ks_2samp

def compute_ks_ppc(real_data, gen_data):
    ks = []
    for col in range(gen_data.shape[1]):
        ks_iter, _ = ks_2samp(real_data, gen_data[:, col])
        ks.append(ks_iter)
    return np.array(ks) 

# 1. Individual Training

## Train the models

In [9]:
zi_vae = VAE(my_dataset.nb_genes, n_batch=my_dataset.n_batches * use_batches, dropout_rate=0.2,
             reconstruction_loss='zinb')
my_zi_trainer = UnsupervisedTrainer(zi_vae, 
                                    my_dataset,
                                    train_size=0.8,
                                    use_cuda=use_cuda,
                                    kl=1, verbose=verbose, frequency=50,
                                    early_stopping_kwargs=early_stopping_kwargs)
my_zi_trainer.train(n_epochs=n_epochs, lr=lr, eps=0.01) 

training:  25%|██▌       | 30/120 [00:15<00:45,  1.99it/s]
Stopping early: no improvement of more than 3 nats in 15 epochs
If the early stopping criterion is too strong, please instantiate it with different parameters in the train method.
training:  26%|██▌       | 31/120 [00:15<00:45,  1.95it/s]


In [ ]:
# train the model
nb_vae = VAE(my_dataset.nb_genes, n_batch=my_dataset.n_batches * use_batches, dropout_rate=0.2,
             reconstruction_loss='nb')
my_nb_trainer = UnsupervisedTrainer(nb_vae, 
                                    my_dataset,
                                    train_size=0.8,
                                    use_cuda=use_cuda, frequency=50,
                                    kl=1,
                                    early_stopping_kwargs=early_stopping_kwargs)
my_nb_trainer.train(n_epochs=n_epochs, lr=lr, eps=0.01)

## PPC for gene expression

#### Zero-inflated

In [ ]:
x_zi_gen, x01 = my_zi_trainer.train_set.generate(genes=None, 
                                                 n_samples=n_samples_posterior_density, zero_inflated=True, batch_size=infer_batch_size)
x_zi_gen = x_zi_gen.squeeze()

#### No zero-inflated 

In [ ]:
x_nb_gen, x02 = my_nb_trainer.train_set.generate(genes=None, n_samples=n_samples_posterior_density, zero_inflated=False, batch_size=infer_batch_size)
x_nb_gen = x_nb_gen.squeeze()

**Warning**: x01 and x02 are supposed to be the same arrays. Histogram plots show that they have the same distributions, however their values are not in the same order. However, statistic tests should be invariant to such an order.

In [ ]:
phi_real_cell = phi(x01, axis=1)
phi_real_gene = phi(x01, axis=0)
phi_zi_gen_cell = phi(x_zi_gen, axis=1)
phi_nb_gen_cell = phi(x_nb_gen, axis=1)
phi_zi_gen_gene = phi(x_zi_gen, axis=0)
phi_nb_gen_gene = phi(x_nb_gen, axis=0)

In [ ]:
ks_zi_cell = compute_ks_ppc(phi_real_cell, phi_zi_gen_cell)
ks_zi_gene = compute_ks_ppc(phi_real_gene, phi_zi_gen_gene)
ks_nb_cell = compute_ks_ppc(phi_real_cell, phi_nb_gen_cell)
ks_nb_gene = compute_ks_ppc(phi_real_gene, phi_nb_gen_gene)

In [ ]:
import seaborn as sns
sns.set()

fig, axes = plt.subplots(nrows=2, sharex=True, figsize=(8, 5))
plt.sca(axes[0])
sns.boxplot(x=ks_zi_cell, color='cyan')
plt.title('Synthetic ZINB')

plt.sca(axes[1])
sns.boxplot(x=ks_nb_cell, color='salmon')
plt.title('Synthetic NB')

fig.suptitle('{} - KS distribution {} - Fixed cell'.format(my_dataset_name, phi_name), fontsize=16)
plt.savefig(os.path.join(save_path, '{}_{}_ks_cell.png'.format(my_dataset_name, phi_name)))

In [ ]:
fig, axes = plt.subplots(nrows=2, sharex=True, figsize=(8, 5))
plt.sca(axes[0])
sns.boxplot(x=ks_zi_gene, color='cyan')
plt.title('Synthetic ZINB')

plt.sca(axes[1])
sns.boxplot(x=ks_nb_gene, color='salmon')
plt.title('Synthetic NB')

fig.suptitle('{} - KS distribution {} - Fixed gene'.format(my_dataset_name, phi_name), fontsize=16)
plt.savefig(os.path.join(save_path, '{}_{}_ks_gene.png'.format(my_dataset_name, phi_name)))

In [ ]:
fig, axes = plt.subplots(nrows=3, sharex=True, figsize=(8, 5))
plt.sca(axes[0])
sns.violinplot(x=phi_real_cell, color='olive')
plt.title('Real Data')

plt.sca(axes[1])
sns.violinplot(x=phi_zi_gen_cell.mean(axis=1), color='cyan')
plt.title('Synthetic ZINB: KS= {0:.6g} +- {1:.6g}'.format(ks_zi_cell.mean() , ks_zi_cell.std()))

plt.sca(axes[2])
sns.violinplot(x=phi_nb_gen_cell.mean(axis=1), color='salmon')
plt.title('Synthetic NB: KS= {0:.6g} +- {1:.6g}'.format(ks_nb_cell.mean(), ks_nb_cell.std()))

fig.suptitle('{} - {} distributions - Fixed cell'.format(my_dataset_name, phi_name), fontsize=16)
plt.savefig(os.path.join(save_path, '{}_{}_phi_cell.png'.format(my_dataset_name, phi_name)))


In [ ]:
fig, axes = plt.subplots(nrows=3, sharex=True, figsize=(8, 5))
plt.sca(axes[0])
sns.violinplot(x=phi_real_gene, color='olive')
plt.title('Real Data')

plt.sca(axes[1])
sns.violinplot(x=phi_zi_gen_gene.mean(axis=1), color='cyan')
plt.title('Synthetic ZINB: KS= {0:.6g} +- {1:.6g}'.format(ks_zi_gene.mean() , ks_zi_gene.std()))

plt.sca(axes[2])
sns.violinplot(x=phi_nb_gen_gene.mean(axis=1), color='salmon')
plt.title('Synthetic NB: KS= {0:.6g} +- {1:.6g}'.format(ks_nb_gene.mean(), ks_nb_gene.std()))

fig.suptitle('{} - {} distributions - Fixed gene'.format(my_dataset_name, phi_name), fontsize=16)
plt.savefig(os.path.join(save_path, '{}_{}_phi_gene.png'.format(my_dataset_name, phi_name)))


# 2. Averaging trainings

In [ ]:
def experiment():
    # Train models
    zi_vae = VAE(my_dataset.nb_genes, n_batch=my_dataset.n_batches * use_batches, dropout_rate=0.2,
                 reconstruction_loss='zinb')
    my_zi_trainer = UnsupervisedTrainer(zi_vae, 
                                        my_dataset,
                                        train_size=0.8,
                                        use_cuda=use_cuda,
                                        kl=1, verbose=verbose, frequency=50,
                                        early_stopping_kwargs=early_stopping_kwargs)
    my_zi_trainer.train(n_epochs=n_epochs, lr=lr, eps=0.01) 

    nb_vae = VAE(my_dataset.nb_genes, n_batch=my_dataset.n_batches * use_batches, dropout_rate=0.2,
             reconstruction_loss='nb')
    my_nb_trainer = UnsupervisedTrainer(nb_vae, 
                                        my_dataset,
                                        train_size=0.8,
                                        use_cuda=use_cuda, frequency=50,
                                        kl=1,
                                        early_stopping_kwargs=early_stopping_kwargs)
    my_nb_trainer.train(n_epochs=n_epochs, lr=lr, eps=0.01)

    # Generate synthetic data
    x_zi_gen, x01 = my_zi_trainer.train_set.generate(genes=None,
                                                     n_samples=n_samples_posterior_density,
                                                     zero_inflated=True,
                                                     batch_size=infer_batch_size)
    x_zi_gen = x_zi_gen.squeeze()

    x_nb_gen, x02 = my_nb_trainer.train_set.generate(genes=None,
                                                     n_samples=n_samples_posterior_density,
                                                     zero_inflated=False,
                                                     batch_size=infer_batch_size)
    x_nb_gen = x_nb_gen.squeeze()
    
    # Compute phi
    phi_real_cell = phi(x01, axis=1)
    phi_real_gene = phi(x01, axis=0)
    phi_zi_gen_cell = phi(x_zi_gen, axis=1)
    phi_nb_gen_cell = phi(x_nb_gen, axis=1)
    phi_zi_gen_gene = phi(x_zi_gen, axis=0)
    phi_nb_gen_gene = phi(x_nb_gen, axis=0)
    
    # Compute KS
    ks_zi_cell = compute_ks_ppc(phi_real_cell, phi_zi_gen_cell)
    ks_zi_gene = compute_ks_ppc(phi_real_gene, phi_zi_gen_gene)
    ks_nb_cell = compute_ks_ppc(phi_real_cell, phi_nb_gen_cell)
    ks_nb_gene = compute_ks_ppc(phi_real_gene, phi_nb_gen_gene)

    return ks_zi_cell, ks_zi_gene, ks_nb_cell, ks_nb_gene

In [ ]:
ks_zi_cell_all, ks_zi_gene_all, ks_nb_cell_all, ks_nb_gene_all = [], [], [], []

for training in tqdm_notebook(range(nb_simu)):
    ks_zi_cell, ks_zi_gene, ks_nb_cell, ks_nb_gene = experiment()
    ks_zi_cell_all.append(ks_zi_cell)
    ks_zi_gene_all.append(ks_zi_gene)
    ks_nb_cell_all.append(ks_nb_cell)
    ks_nb_gene_all.append(ks_nb_gene)
ks_zi_cell_all = np.concatenate(ks_zi_cell_all)
ks_zi_gene_all = np.concatenate(ks_zi_gene_all)
ks_nb_cell_all = np.concatenate(ks_nb_cell_all)
ks_nb_gene_all = np.concatenate(ks_nb_gene_all)

In [ ]:
fig, axes = plt.subplots(nrows=2, sharex=True, figsize=(8, 5))
plt.sca(axes[0])
sns.boxplot(x=ks_zi_gene_all, color='cyan')
plt.title('Synthetic ZINB')

plt.sca(axes[1])
sns.boxplot(x=ks_nb_gene_all, color='salmon')
plt.title('Synthetic NB')

fig.suptitle('{} - KS distribution {} - Fixed gene avg of {} trainings'.format(my_dataset_name, phi_name, nb_simu), fontsize=16)
plt.savefig(os.path.join(save_path, '{}_{}_avg{}_ks_gene.png'.format(my_dataset_name, phi_name, nb_simu)))

In [ ]:
fig, axes = plt.subplots(nrows=2, sharex=True, figsize=(8, 5))
plt.sca(axes[0])
sns.boxplot(x=ks_zi_cell_all, color='cyan')
plt.title('Synthetic ZINB')

plt.sca(axes[1])
sns.boxplot(x=ks_nb_cell_all, color='salmon')
plt.title('Synthetic NB')

fig.suptitle('{} - KS distribution {} - Fixed cell avg of {} trainings'.format(my_dataset_name, phi_name, nb_simu), fontsize=16)
plt.savefig(os.path.join(save_path, '{}_{}_avg{}_ks_cell.png'.format(my_dataset_name, phi_name, nb_simu)))